# Trabalho 2 de Paradigmas da Programação

## Bibliotecas

In [1]:
import sparql_dataframe

## Sparql query

In [2]:
endpoint = "http://dbpedia.org/sparql"

In [3]:
query = """
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
PREFIX dbp: <http://dbpedia.org/property/>
PREFIX dbo: <http://dbpedia.org/ontology/>
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>

select distinct ?nome ?paisNascimento ?dataNascimentoFormatted ?clubeNome ?posicaoLabel ?altura ?numeroCamisa ?allGols ?liga where {
  
  # Pegando as instâncias de SoccerPlayer e o nome do jogador.
  { ?sub rdf:type dbo:SoccerPlayer . }
  union
  { ?sub a dbo:SoccerPlayer . }
  ?sub rdfs:label ?nome .
  FILTER (LANG(?nome) = 'en') .
  
  # Pegando o país de nascimento do jogador.
  ?sub dbo:birthPlace ?localNascimento .
  OPTIONAL { 
    ?localNascimento dbo:country ?paisNascimentoURI . 
  	?paisNascimentoURI rdfs:label ?paisNascimento .
   	FILTER (LANG(?paisNascimento) = 'en') .
  }
  
  # Pegando a data de nascimento do jogador.
  ?sub dbp:birthDate ?dataNascimento .
  
  # Pegando o clube em que o jogador está jogando.
  ?sub dbp:currentclub ?clubeAtual .
  ?clubeAtual rdfs:label ?clubeNome .
  FILTER (LANG(?clubeNome) = 'en') .
  
  # Pegando a posição em que o jogador joga.
  ?sub dbp:position ?posicao .
  ?posicao rdfs:label ?posicaoLabel .
  FILTER (LANG(?posicaoLabel) = 'en') .
  
  # Pegando a altura do jogador.
  ?sub dbo:height ?altura .
  FILTER (datatype(?altura) = xsd:double) .
  
  # Pegando o número da camisa do jogador.
  ?sub dbp:clubnumber ?numeroCamisa .
  FILTER (datatype(?numeroCamisa) = xsd:integer) .
  
  # Pegando a lista de gols do jogador, onde apenas o máximo dessa lista representa a quantidade real de gols dele.
  ?sub dbp:goals ?allGols .
  FILTER (datatype(?allGols) = xsd:integer) .
  
  # Pegando a liga em que o jogador joga.
  ?clubeAtual dbo:league ?ligaURI .
  ?ligaURI rdfs:label ?liga .
  FILTER (LANG(?liga) = 'en') .
  
  
  # Filtrando variáveis que devem ser URIs.
  FILTER (isURI(?clubeAtual)) .
  FILTER (isURI(?paisNascimentoURI)) .
  
  # Transformando as datas de nascimento do formato dateTime para date.
  BIND (
    IF (datatype(?dataNascimento) = xsd:dateTime, STRDT(STR(?dataNascimento), xsd:date), ?dataNascimento)
    AS ?dataNascimentoFormatted
  )
}"""

In [4]:
df = sparql_dataframe.get(endpoint, query)
df.head()

,nome,paisNascimento,dataNascimentoFormatted,clubeNome,posicaoLabel,altura,numeroCamisa,allGols,liga
0,Tirthankar Sarkar,India,1993-07-01,Diamond Harbour FC,Midfielder (association football),1.69,25,0,Calcutta Football League
1,Tirthankar Sarkar,India,1993-07-01,Diamond Harbour FC,Midfielder (association football),1.69,25,1,Calcutta Football League
2,Tirthankar Sarkar,India,1993-07-01,Diamond Harbour FC,Midfielder (association football),1.69,25,2,Calcutta Football League
3,Tirthankar Sarkar,India,1993-07-01,Diamond Harbour FC,Midfielder (association football),1.69,25,7,Calcutta Football League
4,Nub Tola,Cambodia,1995-10-01,Nagaworld FC,Defender (association football),1.75,13,3,Cambodian Premier League


Criando uma coluna a mais no dataframe para conter apenas o ano de nascimento do jogador

In [5]:
def extrair_ano(date):
    return int(date[:4]) if '-' in date else None

In [7]:
df["anoNascimento"] = df["dataNascimentoFormatted"].apply(extrair_ano)
df.head()

,nome,paisNascimento,dataNascimentoFormatted,clubeNome,posicaoLabel,altura,numeroCamisa,allGols,liga,anoNascimento
0,Tirthankar Sarkar,India,1993-07-01,Diamond Harbour FC,Midfielder (association football),1.69,25,0,Calcutta Football League,1993
1,Tirthankar Sarkar,India,1993-07-01,Diamond Harbour FC,Midfielder (association football),1.69,25,1,Calcutta Football League,1993
2,Tirthankar Sarkar,India,1993-07-01,Diamond Harbour FC,Midfielder (association football),1.69,25,2,Calcutta Football League,1993
3,Tirthankar Sarkar,India,1993-07-01,Diamond Harbour FC,Midfielder (association football),1.69,25,7,Calcutta Football League,1993
4,Nub Tola,Cambodia,1995-10-01,Nagaworld FC,Defender (association football),1.75,13,3,Cambodian Premier League,1995


## Regras

### Regra 1

In [8]:
def jogadores_contemporaneos(df):
    pares = [(j1, j2) for j1, ano1 in zip(df['nome'], df['anoNascimento'])
                     for j2, ano2 in zip(df['nome'], df['anoNascimento'])
                     if j1 < j2 and ano1 == ano2]
    return pares

In [9]:
jogadores_contemporaneos(df)

[('Tirthankar Sarkar', 'Vinícius (footballer, born 1993)'),
 ('Tirthankar Sarkar', 'Vinícius (footballer, born 1993)'),
 ('Tirthankar Sarkar', 'Vinícius (footballer, born 1993)'),
 ('Tirthankar Sarkar', 'Vinícius (footballer, born 1993)'),
 ('Tirthankar Sarkar', 'Vinícius (footballer, born 1993)'),
 ('Tirthankar Sarkar', 'Vinícius (footballer, born 1993)'),
 ('Tirthankar Sarkar', 'Vinícius Leite'),
 ('Tirthankar Sarkar', 'Vinícius Leite'),
 ('Tirthankar Sarkar', 'Vinícius Leite'),
 ('Tirthankar Sarkar', 'Vinícius Leite'),
 ('Tirthankar Sarkar', 'Vinícius Leite'),
 ('Tirthankar Sarkar', 'Vinícius Leite'),
 ('Tirthankar Sarkar', 'Vinícius Leite'),
 ('Tirthankar Sarkar', 'Vinícius Leite'),
 ('Tirthankar Sarkar', 'Vinícius Leite'),
 ('Tirthankar Sarkar', 'Vinícius Leite'),
 ('Tirthankar Sarkar', 'Vinícius Leite'),
 ('Tirthankar Sarkar', 'Vinícius Leite'),
 ('Tirthankar Sarkar', 'Zé Roberto (footballer, born 1993)'),
 ('Tirthankar Sarkar', 'Zé Roberto (footballer, born 1993)'),
 ('Tirthanka

### Regra 2

In [13]:
# Função para encontrar jogadores que jogam no mesmo clube
def jogadores_parceiros(df):
    return [(j1, j2) for j1, clube1 in zip(df['nome'], df['clubeNome'])
                     for j2, clube2 in zip(df['nome'], df['clubeNome'])
                     if j1 < j2 and clube1 == clube2]

In [14]:
jogadores_parceiros(df)

[('Nub Tola', 'Sou Yaty'),
 ('Nen Sothearoth', 'Nub Tola'),
 ('Nen Sothearoth', 'Sou Yaty'),
 ('Chhom Pisa', 'Valeriy Hryshyn'),
 ('Chhom Pisa', 'Valeriy Hryshyn'),
 ('Chhom Pisa', 'Valeriy Hryshyn'),
 ('Chhom Pisa', 'Valeriy Hryshyn'),
 ('Chhom Pisa', 'Lim Pisoth'),
 ('Chhom Pisa', 'In Sodavid'),
 ('Seon Power', 'Sor Rotana'),
 ('Seon Power', 'Sor Rotana'),
 ('Robert Lopez Mendy', 'Takashi Odawara'),
 ('Robert Lopez Mendy', 'Takashi Odawara'),
 ('Robert Lopez Mendy', 'Takashi Odawara'),
 ('Robert Lopez Mendy', 'Takashi Odawara'),
 ('Robert Lopez Mendy', 'Víctor Blasco'),
 ('Robert Lopez Mendy', 'Víctor Blasco'),
 ('Robert Lopez Mendy', 'Víctor Blasco'),
 ('Robert Lopez Mendy', 'Víctor Blasco'),
 ('Robert Lopez Mendy', 'Víctor Blasco'),
 ('Robert Lopez Mendy', 'Takashi Odawara'),
 ('Robert Lopez Mendy', 'Takashi Odawara'),
 ('Robert Lopez Mendy', 'Takashi Odawara'),
 ('Robert Lopez Mendy', 'Takashi Odawara'),
 ('Robert Lopez Mendy', 'Víctor Blasco'),
 ('Robert Lopez Mendy', 'Víctor Bla

### Regra 3

In [15]:
# Função para encontrar jogadores super parceiros (mesmo clube, mesmo país, posições diferentes)
def jogadores_super_parceiros(df):
    return [(j1, j2, pais) for j1, clube1, pais, pos1 in zip(df['nome'], df['clubeNome'], df['paisNascimento'], df['posicaoLabel'])
                            for j2, clube2, pais2, pos2 in zip(df['nome'], df['clubeNome'], df['paisNascimento'], df['posicaoLabel'])
                            if j1 < j2 and clube1 == clube2 and pais == pais2 and pos1 != pos2]

In [16]:
jogadores_super_parceiros(df)

[('Nub Tola', 'Sou Yaty', 'Cambodia'),
 ('Nen Sothearoth', 'Sou Yaty', 'Cambodia'),
 ('Chhom Pisa', 'Lim Pisoth', 'Cambodia'),
 ('Chhom Pisa', 'In Sodavid', 'Cambodia'),
 ('Keo Soksela', 'Sa Ty', 'Cambodia'),
 ('Keo Soksela', 'Sa Ty', 'Cambodia'),
 ('Keo Soksela', 'Reung Bunheing', 'Cambodia'),
 ('Keo Soksela', 'Wut Tola', 'Cambodia'),
 ('Keo Soksela', 'Wut Tola', 'Cambodia'),
 ('Keo Soksela', 'Leng Nora', 'Cambodia'),
 ('Keo Soksela', 'Leng Nora', 'Cambodia'),
 ('Keo Soksela', 'Sa Ty', 'Cambodia'),
 ('Keo Soksela', 'Sa Ty', 'Cambodia'),
 ('Keo Soksela', 'Reung Bunheing', 'Cambodia'),
 ('Keo Soksela', 'Wut Tola', 'Cambodia'),
 ('Keo Soksela', 'Wut Tola', 'Cambodia'),
 ('Keo Soksela', 'Leng Nora', 'Cambodia'),
 ('Keo Soksela', 'Leng Nora', 'Cambodia'),
 ('In Sodavid', 'Lim Pisoth', 'Cambodia'),
 ('Leng Nora', 'Sa Ty', 'Cambodia'),
 ('Leng Nora', 'Sa Ty', 'Cambodia'),
 ('Leng Nora', 'Reung Bunheing', 'Cambodia'),
 ('Leng Nora', 'Wut Tola', 'Cambodia'),
 ('Leng Nora', 'Wut Tola', 'Cambodi

### Regra 4

In [18]:
# Função para encontrar centro-avantes goleadores
def centro_avante_goleador(df):
    return df[(df['posicaoLabel'] == 'Forward (association football)') &
              (df['numeroCamisa'] == 9) &
              (df['allGols'] > 50)]

In [ ]:
centro_avante_goleador(df).head()

,nome,paisNascimento,dataNascimentoFormatted,clubeNome,posicaoLabel,altura,numeroCamisa,allGols,liga,anoNascimento
2284,Elkeson,Brazil,1989-07-13,Grêmio Foot-Ball Porto Alegrense,Forward (association football),1.80,9,59,Campeonato Brasileiro Série A,1989
4128,Mersudin Ahmetović,Bosnia & Herzegovina,1985-03-19,FK Sarajevo,Forward (association football),1.91,9,54,Premier League of Bosnia and Herzegovina,1985
6996,Nahikari García,Spain,1997-03-10,Real Madrid Femenino,Forward (association football),1.59,9,104,Primera División (women),1997
7457,Jeison Medina,Colombia,1995-02-27,Al-Shamal SC,Forward (association football),1.80,9,65,Qatar Stars League,1995


### Regra 5

In [20]:
# Função para encontrar jogadores concorrentes (mesmo clube, mesma posição)
def jogadores_concorrentes(df):
    return [(j1, j2, pos, clube) for j1, pos, clube in zip(df['nome'], df['posicaoLabel'], df['clubeNome'])
                                    for j2, pos2, clube2 in zip(df['nome'], df['posicaoLabel'], df['clubeNome'])
                                    if j1 < j2 and pos == pos2 and clube == clube2]

In [21]:
jogadores_concorrentes(df)

[('Nen Sothearoth',
  'Nub Tola',
  'Defender (association football)',
  'Nagaworld FC'),
 ('Seon Power',
  'Sor Rotana',
  'Defender (association football)',
  'Prey Veng FC'),
 ('Seon Power',
  'Sor Rotana',
  'Defender (association football)',
  'Prey Veng FC'),
 ('Sa Ty', 'Wut Tola', 'Forward (association football)', 'Visakha FC'),
 ('Sa Ty', 'Wut Tola', 'Forward (association football)', 'Visakha FC'),
 ('Sa Ty', 'Wut Tola', 'Forward (association football)', 'Visakha FC'),
 ('Sa Ty', 'Wut Tola', 'Forward (association football)', 'Visakha FC'),
 ('Reung Bunheing', 'Sa Ty', 'Forward (association football)', 'Visakha FC'),
 ('Reung Bunheing', 'Sa Ty', 'Forward (association football)', 'Visakha FC'),
 ('Reung Bunheing',
  'Wut Tola',
  'Forward (association football)',
  'Visakha FC'),
 ('Reung Bunheing',
  'Wut Tola',
  'Forward (association football)',
  'Visakha FC'),
 ('Paulo Victor (footballer, born 1994)',
  'Sa Ty',
  'Forward (association football)',
  'Visakha FC'),
 ('Paulo V

### Regra 6

In [22]:
# Função para encontrar jogadores rivais (mesma liga, clubes diferentes)
def jogadores_rivais(df):
    parceiros = jogadores_parceiros(df)
    return [(j1, j2, liga) for j1, clube1, liga in zip(df['nome'], df['clubeNome'], df['liga'])
                              for j2, clube2, liga2 in zip(df['nome'], df['clubeNome'], df['liga'])
                              if j1 < j2 and liga == liga2 and clube1 != clube2
                              and (j1, j2) not in parceiros and (j2, j1) not in parceiros]

In [ ]:
jogadores_rivais(df)

### Regra 7

In [24]:
# Função para encontrar goleiros bons (altura > 1.90 e gols >= 1)
def goleiro_bom(df):
    return df[(df['posicaoLabel'] == 'Goalkeeper (association football)') &
              (df['altura'] >= 1.90) &
              (df['allGols'] >= 1)]

In [25]:
goleiro_bom(df).head()

,nome,paisNascimento,dataNascimentoFormatted,clubeNome,posicaoLabel,altura,numeroCamisa,allGols,liga,anoNascimento
284,"Vinícius (footballer, born 1984)",Brazil,1984-11-09,Clube do Remo,Goalkeeper (association football),1.90,1,1,Campeonato Brasileiro Série C,1984
759,"Everson (footballer, born 1990)",Brazil,1990-07-22,Clube Atlético Mineiro,Goalkeeper (association football),1.92,22,1,Campeonato Mineiro,1990
765,"Everson (footballer, born 1990)",Brazil,1990-07-22,Clube Atlético Mineiro,Goalkeeper (association football),1.92,22,1,Campeonato Brasileiro Série A,1990
1524,Pegorari,Brazil,1991-07-12,Esporte Clube Juventude,Goalkeeper (association football),1.91,1,1,Campeonato Brasileiro Série A,1991
2668,Eduardo Martini,Brazil,1979-01-11,Grêmio Esportivo Brasil,Goalkeeper (association football),1.92,1,1,Campeonato Brasileiro Série D,1979


### Regra 8

In [26]:
# Função para jogadores camisa 10 sem gols
def jogadores_camisa_10_da_shoppe(df):
    return df[(df['numeroCamisa'] == 10) & (df['allGols'] == 0)]

In [27]:
jogadores_camisa_10_da_shoppe(df).head()

,nome,paisNascimento,dataNascimentoFormatted,clubeNome,posicaoLabel,altura,numeroCamisa,allGols,liga,anoNascimento
462,Giovanni Piccolomo,Brazil,1994-04-04,Cruzeiro Esporte Clube,Midfielder,1.73,10,0,Campeonato Mineiro,1994
991,Neilton,Brazil,1994-02-17,Coritiba Foot Ball Club,Forward (association football),1.66,10,0,Campeonato Brasileiro Série A,1994
1122,Yeferson Soteldo,Venezuela,1997-06-30,Santos FC,Midfielder,1.60,10,0,Campeonato Brasileiro Série A,1997
1129,Yeferson Soteldo,Venezuela,1997-06-30,Santos FC,Midfielder,1.60,10,0,Campeonato Paulista,1997
1432,Stiven Mendoza,Colombia,1992-06-27,Ceará Sporting Club,Forward (association football),1.71,10,0,Campeonato Brasileiro Série A,1992
